In [33]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories



In [34]:
import os
from dataclasses import dataclass
from src.datascience.utils.common import read_yaml, create_directories

# Configuration file paths
CONFIG_FILE_PATH = "config/config.yaml"
PARAMS_FILE_PATH = "params.yaml"
SCHEMA_FILE_PATH = "schemas.yaml"

@dataclass
class DataIngestionConfig:
    root_dir: str
    source_url: str
    local_data_file: str
    unzip_dir: str

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schema_filepath))

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig( root_dir=config.root_dir,source_url=config.source_URL, local_data_file=config.local_data_file,unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [35]:
import requests
from pathlib import Path
import urllib.request as request
from src.datascience import logger
import zipfile

In [36]:
## Data Ingestion Componenet
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    #download file from url  
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
            

In [37]:
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion_config = DataIngestion(config=data_ingestion_config)
    data_ingestion_config.download_file()
    data_ingestion_config.extract_zip_file()
except Exception as e:
    raise e

[2025-02-24 17:01:01,665: INFO: common]: yaml file: config/config.yaml loaded successfully
[2025-02-24 17:01:01,669: INFO: common]: yaml file: params.yaml loaded successfully
[2025-02-24 17:01:01,671: INFO: common]: yaml file: schemas.yaml loaded successfully
[2025-02-24 17:01:01,672: INFO: common]: created directory at: artifacts
[2025-02-24 17:01:01,672: INFO: common]: created directory at: artifacts/data_ingestion
[2025-02-24 17:01:05,026: INFO: 3365262817]: artifacts/data_ingestion/winequality-data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3754:68F6D:1BC37:3797C:67BC5878
Accept-Ranges: bytes
Da